# Analysis of dynamic Vicon and EKF noise characteristics 

In [61]:
%load_ext autoreload
%autoreload 2

import scipy
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.spatial.transform import Rotation as R
import loading_utils as lu
import analysis_utils as au
import plotting_utils as pu

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
# Data Loading
state_csv = Path("~/catkin_ws/ros_logs/state_log.csv")
mocap_csv = Path("~/catkin_ws/ros_logs/mocap_log.csv")
state_estimates = lu.QuadStateEstimates.from_csv(state_csv)
vicon_measurements = lu.ViconMeasurements.from_csv(mocap_csv)


No timestamp violations found in state estimates.
No timestamp violations found in Vicon data.


# Vicon Noise Analysis

This section analyzes the noise characteristics of the Vicon motion capture system measurements. We will explore the data, visualize the noise, and discuss its implications for state estimation and sensor fusion.

**Contents:**
- Statistical analysis of noise
- Visualization of position and orientation noise

In [63]:
# --- Position Analysis ---
vicon_stats = au.position_analysis(vicon_measurements.position.to_numpy())
au.print_position_analysis(vicon_stats, label="Vicon Position")

Vicon Position Mean:
 X: -0.0406;    Y: 0.2767;    Z: 0.1052
Vicon Position Std Dev:
 X: 0.00000740;    Y: 0.00000510;    Z: 0.00001010
Vicon Position 95th Percentile:
 X: 0.00001220;    Y: 0.00000824;    Z: 0.00001697
Vicon Position 99th Percentile:
 X: 0.00001736;    Y: 0.00001175;    Z: 0.00002272
Vicon Position Max Deviation:
 X: 0.00002800;    Y: 0.00002299;    Z: 0.00003444
Vicon Position RMS:
 X: 0.00000740;    Y: 0.00000510;    Z: 0.00001010
Vicon Position Covariance Matrix (rows/cols: X, Y, Z):
         X           Y           Z
X  5.4834e-11  1.0015e-11  2.7271e-11
Y  1.0015e-11  2.6023e-11  7.3476e-12
Z  2.7271e-11  7.3476e-12  1.0200e-10


In [64]:
# --- Orientation Analysis ---
vicon_R = R.from_quat(vicon_measurements.orientation, scalar_first=True)
vicon_R_stats = au.orientation_analysis(vicon_R)
au.print_orientation_analysis(vicon_R_stats, label="Vicon Orientation")

Vicon Orientation Mean (as quaternion):
 W: 0.99800;    X: -0.00267;    Y: 0.01854;    Z: 0.06034
Vicon Orientation Std Dev (as deg):
 X: 0.00485678;    Y: 0.00616042;    Z: 0.00160449
Vicon Orientation Vector RMS (as deg):
 X: 0.00485678;    Y: 0.00616042;    Z: 0.00160449
Vicon Orientation Covariance Matrix (rows/cols: x, y, z):
         x           y           z
x  7.1860e-09  -8.1712e-09  -9.8517e-10
y  -8.1712e-09  1.1561e-08  4.8050e-10
z  -9.8517e-10  4.8050e-10  7.8427e-10


# EKF Noise Analysis

In [65]:
# --- Position Analysis ---
ekf_stats = au.position_analysis(state_estimates.position.to_numpy())
au.print_position_analysis(ekf_stats, label="EKF Position")


EKF Position Mean:
 X: -0.0406;    Y: 0.2767;    Z: 0.1052
EKF Position Std Dev:
 X: 0.00000582;    Y: 0.00000453;    Z: 0.00000854
EKF Position 95th Percentile:
 X: 0.00000946;    Y: 0.00000745;    Z: 0.00001419
EKF Position 99th Percentile:
 X: 0.00001412;    Y: 0.00001033;    Z: 0.00001948
EKF Position Max Deviation:
 X: 0.00002179;    Y: 0.00001664;    Z: 0.00002860
EKF Position RMS:
 X: 0.00000582;    Y: 0.00000453;    Z: 0.00000854
EKF Position Covariance Matrix (rows/cols: X, Y, Z):
         X           Y           Z
X  3.3837e-11  3.8622e-12  1.4008e-11
Y  3.8622e-12  2.0512e-11  3.9879e-13
Z  1.4008e-11  3.9879e-13  7.2882e-11


In [66]:
# --- Orientation Analysis ---
ekf_R = R.from_quat(state_estimates.orientation, scalar_first=True)
ekf_R_stats = au.orientation_analysis(ekf_R)
au.print_orientation_analysis(ekf_R_stats, label="EKF Orientation")

EKF Orientation Mean (as quaternion):
 W: 0.99800;    X: -0.00267;    Y: 0.01854;    Z: 0.06034
EKF Orientation Std Dev (as deg):
 X: 0.00493851;    Y: 0.00626142;    Z: 0.00165666
EKF Orientation Vector RMS (as deg):
 X: 0.00493851;    Y: 0.00626142;    Z: 0.00165666
EKF Orientation Covariance Matrix (rows/cols: x, y, z):
         x           y           z
x  7.4315e-09  -8.3746e-09  -1.0322e-09
y  -8.3746e-09  1.1946e-08  4.7159e-10
z  -1.0322e-09  4.7159e-10  8.3628e-10


In [67]:
# --- Linear Velocity Analysis ---
# Given that the Quad is static, we can analyze the velocity estimates as they were position measurements.
ekf_vel_stats = au.position_analysis(state_estimates.velocity_linear.to_numpy())
au.print_position_analysis(ekf_vel_stats, label="EKF Velocity")

EKF Velocity Mean:
 X: 0.0000;    Y: 0.0000;    Z: 0.0000
EKF Velocity Std Dev:
 X: 0.00019275;    Y: 0.00016145;    Z: 0.00030768
EKF Velocity 95th Percentile:
 X: 0.00031689;    Y: 0.00026885;    Z: 0.00051520
EKF Velocity 99th Percentile:
 X: 0.00045698;    Y: 0.00037544;    Z: 0.00069897
EKF Velocity Max Deviation:
 X: 0.00065459;    Y: 0.00051459;    Z: 0.00103181
EKF Velocity RMS:
 X: 0.00019275;    Y: 0.00016145;    Z: 0.00030768
EKF Velocity Covariance Matrix (rows/cols: X, Y, Z):
         X           Y           Z
X  3.7165e-08  5.7448e-09  1.7401e-08
Y  5.7448e-09  2.6073e-08  -6.6419e-09
Z  1.7401e-08  -6.6419e-09  9.4693e-08


In [68]:
# --- Linear Acceleration Analysis ---
# Given that the Quad is static, we can analyze the linear acceleration estimates as they were position measurements.
ekf_acc_stats = au.position_analysis(state_estimates.acceleration_linear.to_numpy())
au.print_position_analysis(ekf_acc_stats, label="EKF Linear Acceleration")

EKF Linear Acceleration Mean:
 X: 0.0000;    Y: 0.0000;    Z: 0.0000
EKF Linear Acceleration Std Dev:
 X: 0.00152731;    Y: 0.00128360;    Z: 0.00244574
EKF Linear Acceleration 95th Percentile:
 X: 0.00250580;    Y: 0.00211907;    Z: 0.00412004
EKF Linear Acceleration 99th Percentile:
 X: 0.00358214;    Y: 0.00296279;    Z: 0.00547322
EKF Linear Acceleration Max Deviation:
 X: 0.00512871;    Y: 0.00419481;    Z: 0.00838427
EKF Linear Acceleration RMS:
 X: 0.00152731;    Y: 0.00128360;    Z: 0.00244574
EKF Linear Acceleration Covariance Matrix (rows/cols: X, Y, Z):
         X           Y           Z
X  2.3334e-06  3.6421e-07  1.0856e-06
Y  3.6421e-07  1.6481e-06  -4.2960e-07
Z  1.0856e-06  -4.2960e-07  5.9834e-06


In [69]:
# --- Angular Velocity Analysis ---
# Given that the Quad is static, we can analyze the angular velocity estimates as they were position measurements.
ekf_ang_vel_stats = au.position_analysis(state_estimates.velocity_angular.to_numpy())
au.print_position_analysis(ekf_ang_vel_stats, label="EKF Angular Velocity")

print(f"Maximum body rate deviation (deg/s): \n X: {np.rad2deg(ekf_ang_vel_stats.max_deviation[0]):.5f};   Y: {np.rad2deg(ekf_ang_vel_stats.max_deviation[1]):.5f};   Z: {np.rad2deg(ekf_ang_vel_stats.max_deviation[2]):.5f};" )

EKF Angular Velocity Mean:
 X: 0.0000;    Y: -0.0000;    Z: 0.0000
EKF Angular Velocity Std Dev:
 X: 0.00512194;    Y: 0.00811427;    Z: 0.00191860
EKF Angular Velocity 95th Percentile:
 X: 0.00798774;    Y: 0.01345704;    Z: 0.00318530
EKF Angular Velocity 99th Percentile:
 X: 0.01048987;    Y: 0.01740311;    Z: 0.00446355
EKF Angular Velocity Max Deviation:
 X: 0.01523307;    Y: 0.02189308;    Z: 0.00657795
EKF Angular Velocity RMS:
 X: 0.00512194;    Y: 0.00811427;    Z: 0.00191860
EKF Angular Velocity Covariance Matrix (rows/cols: X, Y, Z):
         X           Y           Z
X  2.6242e-05  -3.5954e-05  -3.8710e-06
Y  -3.5954e-05  6.5861e-05  1.7048e-06
Z  -3.8710e-06  1.7048e-06  3.6821e-06
Maximum body rate deviation (deg/s): 
 X: 0.87279;   Y: 1.25438;   Z: 0.37689;


# Pseudo-Innovation Analysis
- Using the EKF to Vicon residuals

In [83]:
# --- Position Interpolation ---
vicon_positions_interp , valid_interp_idx=au.interpolate_position(vicon_measurements.position.to_numpy(), vicon_measurements.time.to_numpy(), state_estimates.time.to_numpy())
efk_positions_interp = state_estimates.position.to_numpy()[valid_interp_idx]
efk_to_vicon_positions = efk_positions_interp - vicon_positions_interp
# --- Position Analysis ---
ekf_to_vicon_stats = au.position_analysis(efk_to_vicon_positions)
au.print_position_analysis(ekf_to_vicon_stats, label="EKF to Vicon Position")


EKF to Vicon Position Mean:
 X: 0.0000;    Y: 0.0000;    Z: 0.0000
EKF to Vicon Position Std Dev:
 X: 0.00002449;    Y: 0.00004718;    Z: 0.00007559
EKF to Vicon Position 95th Percentile:
 X: 0.00001197;    Y: 0.00000785;    Z: 0.00001535
EKF to Vicon Position 99th Percentile:
 X: 0.00001746;    Y: 0.00001182;    Z: 0.00002256
EKF to Vicon Position Max Deviation:
 X: 0.00135558;    Y: 0.00272390;    Z: 0.00435082
EKF to Vicon Position RMS:
 X: 0.00002449;    Y: 0.00004718;    Z: 0.00007559
EKF to Vicon Position Covariance Matrix (rows/cols: X, Y, Z):
         X           Y           Z
X  6.0001e-10  1.1055e-09  1.7766e-09
Y  1.1055e-09  2.2267e-09  3.5150e-09
Z  1.7766e-09  3.5150e-09  5.7161e-09


In [84]:
# --- Orientation Interpolation ---
vicon_orientation_interp, valid_interp_idx = au.interpolate_orientation(R.from_quat(vicon_measurements.orientation.to_numpy(), scalar_first=True), vicon_measurements.time.to_numpy(), state_estimates.time.to_numpy())
ekf_to_vicon_R = R.from_quat(state_estimates.orientation.to_numpy()[valid_interp_idx], scalar_first=True) * vicon_orientation_interp.inv()

# --- Orientation Analysis ---
ekf_to_vicon_R_stats = au.orientation_analysis(ekf_to_vicon_R)
au.print_orientation_analysis(ekf_to_vicon_R_stats, label="EKF to Vicon Orientation")

EKF to Vicon Orientation Mean (as quaternion):
 W: 1.00000;    X: -0.00000;    Y: -0.00000;    Z: 0.00000
EKF to Vicon Orientation Std Dev (as deg):
 X: 0.00620753;    Y: 0.00787893;    Z: 0.00207499
EKF to Vicon Orientation Vector RMS (as deg):
 X: 0.00620753;    Y: 0.00787893;    Z: 0.00207499
EKF to Vicon Orientation Covariance Matrix (rows/cols: x, y, z):
         x           y           z
x  1.1741e-08  -1.3243e-08  -1.5583e-09
y  -1.3243e-08  1.8915e-08  6.4181e-10
z  -1.5583e-09  6.4181e-10  1.3119e-09


# Vicon Noise Characteristics

In [ ]:
acf_x = au.compute_acf(vicon_measurements.position.to_numpy()[:,0])
acf_y = au.compute_acf(vicon_measurements.position.to_numpy()[:,1])
acf_z = au.compute_acf(vicon_measurements.position.to_numpy()[:,2])

# pu.plot_acf_plotly(acf_x, title="X ACF with 95% bounds")
au.print_acf_analysis(acf_x)


=== ACF Analysis Summary ===
95% confidence band under white-noise null: [-0.0158, 0.0158]
Total lags checked: 2000 (excluding lag=0)
Significant lags: 838

Top 10 significant lags:
  Lag   0: ACF=+1.0000
  Lag   1: ACF=+0.2624
  Lag   3: ACF=+0.2166
  Lag   2: ACF=+0.2157
  Lag   4: ACF=+0.2100
  Lag  46: ACF=+0.1923
  Lag   5: ACF=+0.1922
  Lag   6: ACF=+0.1799
  Lag   7: ACF=+0.1577
  Lag   8: ACF=+0.1529


# EKF Noise Characteristics

In [ ]:
acf_x = au.compute_acf(state_estimates.position.to_numpy()[:,0])
acf_y = au.compute_acf(state_estimates.position.to_numpy()[:,1])
acf_z = au.compute_acf(state_estimates.position.to_numpy()[:,2])

# pu.plot_acf_plotly(acf_x, title="X ACF with 95% bounds")

au.print_acf_analysis(acf_x)



=== ACF Analysis Summary ===
95% confidence band under white-noise null: [-0.0287, 0.0287]
Total lags checked: 1165 (excluding lag=0)
Significant lags: 528

Top 10 significant lags:
  Lag   0: ACF=+1.0000
  Lag   1: ACF=+0.5830
  Lag   2: ACF=+0.3010
  Lag  17: ACF=+0.2381
  Lag  51: ACF=+0.2378
  Lag  18: ACF=+0.2240
  Lag  19: ACF=+0.2010
  Lag  50: ACF=+0.1888
  Lag  52: ACF=+0.1872
  Lag  16: ACF=+0.1776
